In [9]:
import os
import re
import random
import copy
from collections import Counter
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gc
    
from gensim.models import KeyedVectors

import numpy as np
from numpy.random import seed
seed(1000)
from tensorflow.random import set_random_seed
set_random_seed(1000)

import pandas
from pandas import Series, DataFrame

import keras
import keras.backend as K
# from keras.layers import Layer
from keras.engine import Layer
from keras.engine import InputSpec
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.layers import Input, Embedding, Conv1D, Concatenate, Flatten, Dropout, Dense, Bidirectional, Activation
from keras.layers import MaxPooling1D, Conv2D, Flatten, Reshape, MaxPooling2D, LSTM, Lambda, Highway, Dot, Permute, Add
from keras.models import Model, Sequential
from keras.activations import softmax
from keras.initializers import Constant, RandomNormal, RandomUniform

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

import matplotlib.pyplot as plt
import seaborn as sns       

from scipy import stats
from scipy.stats import  norm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

#进行配置，使用30%的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.Session(config=config)

# 设置session
KTF.set_session(session )

In [2]:
# 文件路径
ORIGINAL_FILE_PATH = "../data/original_data/quora_duplicate_questions.tsv"

DATA_ALL_PATH = "../data/all_data.txt"

DATA_TRAIN_PATH = "../data/train.txt"
DATA_VALID_PATH = "../data/valid.txt"
DATA_TEST_PATH = "../data/test.txt"

MAPS_FILE_PATH = "../data/maps.pkl"

# Glove文件
WORD2VEC_PATH = "../data/GoogleNews-vectors-negative300.bin"

# Model Path
MODEL_PATH = "../model/duet/duet-01.h5"

# 创建文件夹
(file_path, file_name) = os.path.split(MODEL_PATH)
if not os.path.exists(file_path):
    os.mkdir(file_path)

In [3]:
def load_dict(dict_path):
    with open(dict_path, 'rb') as fr:
        return pickle.load(fr)
        
def create_maps(dico):
    item2id = dico
    id2item = dict([val, key] for key, val in dico.items())
    return item2id, id2item

def read_file(fpath):
    """
    读取文件，返回
    """
    question1, question2, labels = [], [], []
    with open(fpath, 'r', encoding="utf-8") as fr:
        for line in fr.readlines():
            try:
                q, k, l = line.strip().split('\t')
            except ValueError:
                print(line)
            question1.append(q)
            question2.append(k)
            labels.append(int(l))
            
    return question1, question2, labels

def load_data(fpath, word_to_id, max_length=20):
    """
    载入数据，并将其转化为id表示
    """
    question1, question2, labels = read_file(fpath)
    
    q1_id, q2_id = [], []
    for i in range(len(question1)):
        q1= question1[i].split()
        q2 = question2[i].split()
        q1_id.append([word_to_id[x] for x in q1 if x in word_to_id])
        q2_id.append([word_to_id[x] for x in q2 if x in word_to_id])

    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x1_pad = keras.preprocessing.sequence.pad_sequences(q1_id, max_length, padding="post")
    x2_pad = keras.preprocessing.sequence.pad_sequences(q2_id, max_length, padding="post")
    labels = keras.utils.np_utils.to_categorical(labels)
    
    return x1_pad, x2_pad, labels
    
        
def load_word2vec(embedding_path, word_index, embed_dim=300): 
    """
    载入预训练好的word2vec(Google-News)
    """
    # 载入word2vec词向量
    word2vec_dict = KeyedVectors.load_word2vec_format(embedding_path,binary=True)
    
    embedding_index = dict()
    for word in word2vec_dict.wv.vocab:
        embedding_index[word] = word2vec_dict.word_vec(word)
    print('Load %s word vectors.' % len(embedding_index))
    
    all_embs = np.stack(list(embedding_index.values()))
    # emb_mean, emb_std = all_embs.mean(), all_embs.std()
    emb_mean = np.mean(all_embs, axis=0)
    vocab_size = len(word_index)
    # 初始化权重
    embedding_matrix = np.zeros((vocab_size+1, embed_dim))
    gc.collect()
    # 对权重矩阵进行赋值，未找到时用词向量平均值填充
    for word, i in word_index.items():
        if i < vocab_size + 1:
            embedding_vector = embedding_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
            else:
                embedding_matrix[i] = emb_mean
    del embedding_index
    return embedding_matrix

In [4]:
dico = load_dict(MAPS_FILE_PATH)
word_index, index_word = create_maps(dico)

# 准备数据
x1_train, x2_train, y_train = load_data(DATA_TRAIN_PATH, word_index)
x1_valid, x2_valid, y_valid = load_data(DATA_VALID_PATH, word_index)
x1_test, x2_test, y_test = load_data(DATA_TEST_PATH, word_index)

embedding_matrix = load_word2vec(WORD2VEC_PATH, word_index)

Load 3000000 word vectors.


In [6]:
vocab_size = len(dico)

class Settings(object):
    text1_maxlen = 20
    text2_maxlen = 20
    embedding_dim = 300
    vocab_size = vocab_size
    
    lm_kernel_count = 32
    lm_hidden_sizes = [50]
    dm_kermel_count = 32
    dm_kernel_size = 3
    dm_q_hidden_size = 32
    dm_d_mpool = 3
    dm_hidden_sizes = [50]
    lm_dropout_rate = 0.2
    dm_dropout_rate = 0.2
    
    dropout_rate = 0.2
    
settings = Settings()

In [12]:
# 定义网络
def xor_match(x):
    t1 = x[0]
    t2 = x[1]
    t1_shape = t1.get_shape()
    t2_shape = t2.get_shape()
    t1_expand = K.tf.stack([t1] * t2_shape[1], 2)
    t2_expand = K.tf.stack([t2] * t1_shape[1], 1)
    out_bool = K.tf.equal(t1_expand, t2_expand)
    out = K.tf.cast(out_bool, K.tf.float32)
    return out
def hadamard_dot(x):
    x1 = x[0]
    x2 = x[1]
    out = x1 * x2
    return out

query = Input(name='query', shape=(settings.text1_maxlen,))
doc = Input(name='doc', shape=(settings.text2_maxlen, ))


embedding = Embedding(settings.vocab_size + 1,
                      settings.embedding_dim,
                      weights=[embedding_matrix],
                      input_length=settings.text1_maxlen,
                      trainable=False)

q_embed = embedding(query)
d_embed = embedding(doc)

lm_xor = Lambda(xor_match)([query, doc])
lm_conv = Conv1D(settings.lm_kernel_count, settings.text2_maxlen, padding='same', activation='tanh')(lm_xor)
lm_conv = Dropout(settings.lm_dropout_rate)(lm_conv)
lm_feat = Reshape((-1,))(lm_conv)
for hidden_size in settings.lm_hidden_sizes:
    lm_feat = Dense(hidden_size, activation='tanh')(lm_feat)

lm_drop = Dropout(settings.lm_dropout_rate)(lm_feat)
lm_score = Dense(1)(lm_drop)
dm_q_conv = Conv1D(settings.dm_kermel_count, settings.dm_kernel_size, padding='same', activation='tanh')(q_embed)
dm_q_conv = Dropout(settings.dm_dropout_rate)(dm_q_conv)
dm_q_mp = MaxPooling1D(pool_size = settings.text1_maxlen)(dm_q_conv)        
dm_q_rep = Reshape((-1,))(dm_q_mp)
dm_q_rep = Dense(settings.dm_q_hidden_size)(dm_q_rep)
dm_q_rep = Lambda(lambda x: tf.expand_dims(x, 1))(dm_q_rep)    
dm_d_conv1 = Conv1D(settings.dm_kermel_count, settings.dm_kernel_size, padding='same', activation='tanh')(d_embed)        
dm_d_conv1 = Dropout(settings.dm_dropout_rate)(dm_d_conv1)
dm_d_mp = MaxPooling1D(pool_size = settings.dm_d_mpool)(dm_d_conv1)      
dm_d_conv2 = Conv1D(settings.dm_kermel_count, 1, padding='same', activation='tanh')(dm_d_mp)
dm_d_conv2 = Dropout(settings.dm_dropout_rate)(dm_d_conv2)
h_dot = Lambda(hadamard_dot)([dm_q_rep, dm_d_conv2])        
dm_feat = Reshape((-1,))(h_dot)
for hidden_size in settings.dm_hidden_sizes:
    dm_feat = Dense(hidden_size)(dm_feat)
        
dm_feat_drop = Dropout(settings.dm_dropout_rate)(dm_feat)        
       
dm_score = Dense(1)(dm_feat_drop)
out_ = Add()([lm_score, dm_score])

out_ = Dense(2, activation='softmax')(out_)

model = Model(inputs=[query, doc], outputs=out_)
model.summary()        

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query (InputLayer)              (None, 20)           0                                            
__________________________________________________________________________________________________
doc (InputLayer)                (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 20, 300)      25293600    query[0][0]                      
                                                                 doc[0][0]                        
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 20, 32)       28832       embedding_4[0][0]                
__________

In [13]:
import datetime,time
from keras.optimizers import Adam

callback_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc',
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=MODEL_PATH,
        monitor='val_acc',
        save_best_only=True
    )
]

adadelta=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['binary_accuracy'])
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

print('Train...')

model.fit([x1_train, x2_train], y_train, epochs=100, batch_size=128, callbacks=callback_list, validation_data=([x1_valid, x2_valid], y_valid))

Train...
Train on 323426 samples, validate on 40428 samples
Epoch 1/100
323426/323426 [==============================] - 31s 95us/step - loss: 0.5010 - acc: 0.7413 - val_loss: 0.4572 - val_acc: 0.7696
Epoch 2/100
323426/323426 [==============================] - 28s 87us/step - loss: 0.4502 - acc: 0.7753 - val_loss: 0.4364 - val_acc: 0.7837
Epoch 3/100
323426/323426 [==============================] - 28s 87us/step - loss: 0.4304 - acc: 0.7887 - val_loss: 0.4322 - val_acc: 0.7911
Epoch 4/100
323426/323426 [==============================] - 28s 87us/step - loss: 0.4168 - acc: 0.7972 - val_loss: 0.4169 - val_acc: 0.7975
Epoch 5/100
323426/323426 [==============================] - 28s 87us/step - loss: 0.4062 - acc: 0.8037 - val_loss: 0.4124 - val_acc: 0.8006
Epoch 6/100
323426/323426 [==============================] - 28s 87us/step - loss: 0.3982 - acc: 0.8085 - val_loss: 0.4351 - val_acc: 0.7899
Epoch 7/100
323426/323426 [==============================] - 28s 87us/step - loss: 0.3907 - ac

In [14]:
y_vec_pred = model.predict([x1_test, x2_test], batch_size=128)

In [15]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

def to_one_hot(prob):
    prob_shape = prob.shape
    y_pred = np.zeros(prob_shape)
    
    i = 0
    for item in prob:
        if item[0] < item[1]:
            y_pred[i] = np.array([0, 1])
        else:
            y_pred[i] = np.array([1, 0])
        i += 1
    return y_pred

y_pred = to_one_hot(y_vec_pred)
print(y_pred)

[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [16]:
report = precision_recall_fscore_support(y_test, y_pred, average='macro')
    
def print_prf(result):
    precision = str(round(result[0] * 100, 2)) + "%"
    recall = str(round(result[1] * 100, 2)) + "%"
    f1 = str(round(result[2] * 100, 2)) + "%"
    
    strResult="Precision:"+precision+"\tRecall:"+recall+"\tF-score:"+f1
    print(strResult)
    
print_prf(report)

Precision:78.46%	Recall:79.99%	F-score:78.86%


In [17]:
model.evaluate([x1_test, x2_test], y_test)

40429/40429 [==============================] - 4s 87us/step


[0.47495094006041916, 0.7964085186420345]